# Code for calculating number of distinct genes found in the plass assemblies of neighborhoods

In [5]:
import screed, collections

In [9]:
def do_mincount(filename):
    by_genome = collections.defaultdict(list)

    records = list(screed.open(filename))
    
    for record in records:
        genome = record.name.split('_')[0]
        by_genome[genome].append(record.sequence)
        
    maxlen = len(record.sequence)

    zz = []

    for name in by_genome:
        # for each position in the alignment, find all aligned non-gap characters
        by_pos = [ set() for x in range(maxlen) ]
        for sequence in by_genome[name]:
            for pos, ch in enumerate(sequence):
                if ch != '-':
                    by_pos[pos].add(ch)

        # count how many positions have 1 character, 2 characters, etc.
        c = collections.Counter()
        for x in by_pos:
            c[len(x)] += 1

        # now eliminate anything that has fewer than 10 supporting positions
        xx = c.most_common()
        xx = [ (a, b) for (a, b) in xx if b > 10 ]

        # find the biggest number of well-supported distinct characters
        xx.sort()
        zz.append((name, xx[-1][0]))

    zz = dict(zz)

    xx = []
    for i in range(19, 42):
        bin_num = zz.get('BIN-hu-genome{}'.format(i), 0)
        nbhd_num = zz.get('hu-genome{}'.format(i), 0)
        xx.append((i, bin_num, nbhd_num))

    return xx
                   
gyrA = do_mincount('plass-hardtrim-all-bin-PF00521_gyra-hmmscanT100-mafft.faa')
print(gyrA)

[(19, 2, 0), (20, 0, 2), (21, 0, 2), (22, 0, 0), (23, 1, 1), (24, 1, 2), (25, 2, 3), (26, 1, 1), (27, 1, 2), (28, 0, 0), (29, 0, 0), (30, 0, 2), (31, 0, 0), (32, 0, 1), (33, 1, 1), (34, 1, 2), (35, 1, 1), (36, 2, 2), (37, 1, 1), (38, 1, 1), (39, 1, 1), (40, 1, 2), (41, 1, 2)]


In [10]:
recA = do_mincount('plass-hardtrim-all-bin-PF00154_reca-hmmscanT100-mafft.faa')
rplB = do_mincount('plass-hardtrim-all-bin-PF00181_rplb-hmmscanT100-mafft.faa')
rpsC = do_mincount('plass-hardtrim-all-bin-PF00189_rpsc-hmmscanT100-mafft.faa')
gyrB = do_mincount('plass-hardtrim-all-bin-PF00204_gyrb-hmmscanT100-mafft.faa')
gyrA = do_mincount('plass-hardtrim-all-bin-PF00521_gyra-hmmscanT100-mafft.faa')
rpb = do_mincount('plass-hardtrim-all-bin-PF00562_rpb2d6-hmmscanT100-mafft.faa')
alaS = do_mincount('plass-hardtrim-all-bin-PF01411_alas-hmmscanT100-mafft.faa')


## output latex for the summary spreadsheets

In [12]:
import pandas
names_df = pandas.read_csv('TR-hu-sb1-plass-gyrA.csv')

names_d = {}
for row in names_df.itertuples():
    names_d[row.name] = row.abbreviation

In [14]:
import sys
def latex_output(mincounts, fp=sys.stdout):
    for row in sorted(mincounts, key=lambda x:(x[1], x[2], x[0])):
        num, orig, mincount = row
        name = 'hu-genome{}'.format(num)
        abbreviation = names_d[name]

        print(abbreviation.replace('_', '\\_'), '&', orig, '&', mincount, '\\\\', file=fp)

latex_output(recA, open('recA-table.tex', 'wt'))
latex_output(rplB, open('rplB-table.tex', 'wt'))
latex_output(rpsC, open('rpsC-table.tex', 'wt'))
latex_output(gyrB, open('gyrB-table.tex', 'wt'))
latex_output(gyrA, open('gyrA-table.tex', 'wt'))
latex_output(rpb, open('rpb-table.tex', 'wt'))
latex_output(alaS, open('alaS-table.tex', 'wt'))
